In [964]:
import cv2
import os
import csv
import numpy as np
import math
from scipy import ndimage
from matplotlib import pyplot as plt #to make visualization easier

In [965]:
def contours(image_paths, canny):
    #apply contours and generate image
    contours, _ = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    img = cv2.imread(image_paths['input'])
    # new_img = cv2.drawContours(img, contours, -1, (0,255,0), 1)

    # output_filename = f"{image_paths['scan_dir']}/contours.png"
    # cv2.imwrite(output_filename, new_img)
    return contours


In [966]:
def get_angle_simple(a, b):
    
    #Calculates the angle of a straight line (with respect to the horizon) between points a and b:
    return math.degrees(math.atan2(a[1] - b[1], a[0] - b[0]))

In [967]:
def match_template(result, img, template, template_name, threshold):
    #cross correlation method is more efficient 
    #Match template based on threshold value, draw red rectangle over templates, 
    #INPUTS: image to draw on, image to perform template match on, template, name of template (only used in print statement), threshold of accuracy
    # OUTPUTS: alignment pts array, resulting image with red rectangle drawn over templates:
    w, h = template.shape[::-1]
    
    #Using matching operation TM_CCORR_NORMED (23/01/30):
    #spits out a matrix of numbers btwn 0-1 correpsonding to each pixel's match
    res = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    
    #counts number of nonzeroes that is greater than threshold for template matching
    n = np.count_nonzero(res >= threshold)
    #turns matrix into array 
    pts = np.dstack(np.unravel_index(np.argsort(res.ravel()), res.shape))[0]
    pts = pts[::-1][:n] #reverses order of elements of pts array so points with higher corr coeff are at top of 2d array, then selects first n elements of array
    pts[:, [1, 0]] = pts[:, [0, 1]]
    
    delete = []
    for i in range(len(pts)):
        for j in range(i + 1, len(pts)):
            if np.linalg.norm(pts[i] - pts[j]) <= 15 and j not in delete:
                delete.append(j)
    pts = np.delete(pts, delete, axis=0)
    #sorts lexographically, with sorts based on the second column (index 1), and then for elements with equal values in the second column, it sorts based on the first column (index 0). This results in an array of indices indicating the sorted order of the original array.
    pts = pts[np.lexsort((pts[:, 0], pts[:, 1]))]
    
#draws a rectangle around each template in the result image
    for pt in pts:
        #inputs: top left coord of rect, bottom right coord, color(BGR), thickness of rectangle in pixels
        cv2.rectangle(result, (pt[0], pt[1]), (pt[0] + w, pt[1] + h), (0, 0, 255), 8)
        
    print("Detected " + template_name + " markers:", len(pts))
    
    return pts, result

In [968]:
def truncate_contour(cross_pts, square_pts, canny, template, image_paths, output):
    #inputs: corner alignment pts, canny image, template to get w,h to form square that will truncate off image
    #outputs: image with contours, contours
    #finds corner alignment contours by cutting out part of image that contains the cross with template matching
    w, h = template.shape[::-1]
    cross_contours = []
    for i in range(len(cross_pts)):
        #take region of interest in canny image
        roi = canny[cross_pts[i][1]:cross_pts[i][1]+h, cross_pts[i][0]:cross_pts[i][0]+w]
        #will always contain 1 contour (most of the time)
        contours, _ = cv2.findContours(roi, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
        
        # Shift contour coordinates by the offset of the cutout region
        offset_x, offset_y = cross_pts[i]  # Unpack offset coordinates
        cnt = max(contours, key=cv2.contourArea) #take the contour with the biggest area in the array of detected contours
        # Create new contour with shifted points
        shifted_cnt = cnt.copy()  # Copy the original contour
        # Shift points using broadcasting
        shifted_cnt += np.array([[offset_x, offset_y]])
        cross_contours.append(shifted_cnt.astype(np.int32))

    
    square_contours = []
    square_contour = []
    for i in range(len(square_pts)):
        #take region of interest in canny image
        roi = canny[square_pts[i][1]-50:square_pts[i][1]+h+50, square_pts[i][0]-50:square_pts[i][0]+w+50]
        cv2.rectangle(output, (square_pts[i][0]-50, square_pts[i][1]-50), (square_pts[i][0]+w+50, square_pts[i][1]+h+50), (255,255,0), 3)
        #will always contain 4 contours, for each square, have to do RETR_EXTERNAL to get outermost contours in hierarchy
        contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
        offset_x, offset_y = square_pts[i][0]-50,square_pts[i][1]-50  # Unpack offset coordinates
        
        for c in contours:
            c += np.array([offset_x, offset_y])
            if(200 < cv2.contourArea(c) < 300):
                square_contours.append(c)
                cv2.drawContours(output, c, -1, (0, 255, 0), 3)
        copy = square_contours.copy()#for some reason I have to make a copy or the square_contour will have an empty square_contours appended to it
        square_contour.append(copy)
        square_contours.clear()
        
    
   
    #filter through contours and only keep the ones that have a proper area

        # #adds each element of the contours list as individual elements to square_contours instead of append, which adds entire list of contours as one element
        # square_contours.extend(contours)

    
    # #for every 4 contours, add the offset of the upper left template matched location
    # i=0
    # print("length of contours", len(square_contours))
    # print("length of template match single point:", len(square_pts))
    # for c in range(0,len(square_contours), 4):
    #     # Shift contour coordinates by the offset of the cutout region
    #     offset_x, offset_y = square_pts[i]  # Unpack offset coordinates
        
    #     square_contours[c] += np.array([offset_x, offset_y])
    #     square_contours[c+1] += np.array([offset_x, offset_y])
    #     square_contours[c+2]+= np.array([offset_x, offset_y])
    #     square_contours[c+3] += np.array([offset_x, offset_y])
    #     i+=1
    # print(square_contours)

            
       
    # i=0
    # area = []
    # for c in shifted_cnt:
    #        # Shift contour coordinates by the offset of the cutout region
    #      area.append((cv2.contourArea(c),c))
         
    # area.sort(key=lambda x: x[0], reverse=True)

    
    
    new_img = cv2.drawContours(output, cross_contours, -1, (0, 255, 0), 3)
    final = cv2.drawContours(new_img, square_contours, -1, (0, 255, 0), 3)
    return final, cross_contours, square_contour

In [969]:
def centroid(die_contours, cross_contours, square_contour, alignment_pts, image_paths, ideal_param, out):
    cross_centers = []
    square_centers = []
    shifts = {}
    shifts_csv = {}
    for c in cross_contours:
        # calculate moments for each contour
        M = cv2.moments(c)
        
        # calculate x,y coordinate of center
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        point = [cX, cY]
        cross_centers.append(point)
        cv2.circle(out, (cX, cY), 5, (255, 0, 0), -1)
        cv2.putText(out, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    centerlist = []
    for contourlist in square_contour:
        four_centers = []
        
        for c in contourlist:
            M1 = cv2.moments(c)
            cX = int(M1["m10"] / M1["m00"])
            cY = int(M1["m01"] / M1["m00"])
            point = [cX, cY]
            four_centers.append(point)
           
            cv2.circle(out, (cX, cY), 5, (255, 0, 0), -1)
            # cv2.putText(out, "square center", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                
        copy = four_centers.copy()
        centerlist.append(copy)
        four_centers.clear()
    
   
    for list in centerlist:
        center_x = []
        center_y = []
        for x, y in list:
            center_x.append(x)
            center_y.append(y)
        x = np.mean(center_x)
        y = np.mean(center_y)
        print("center", (x,y))
        square_centers.append((x,y))
     

        
    
    for p in square_centers:
        print("centroid", p)
        x, y = p
        if math.isnan(x) or math.isnan(y):
            continue
        x = int(x)
        y = int(y)
        cv2.circle(out, (x, y), 5, (180, 105, 255), -1)
        cv2.putText(out, "centroid", (x - 25, y - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (180, 105, 255), 2)

    #calculate and draw center of die
    die_centers = []
    row=0
    col=0
    for c in die_contours:
       
       center_inside = []
       for i in range(len(square_centers)):
        inside = cv2.pointPolygonTest(c, square_centers[i], False)
        if(inside == 1):
           center_inside.append(square_centers[i]) #should add 2 centers to center_inside in order by die
       for i in range(len(cross_centers)):
        inside = cv2.pointPolygonTest(c, cross_centers[i], False)
        if(inside == 1):
           center_inside.append(cross_centers[i]) #should add 2 centers to center_inside in order by die
       die_center = np.mean(center_inside, axis=0)
       ox = die_center[0]
       oy = die_center[1]

       shifts_csv[str(row) + ', ' + str(col)] = {}
       shifts_csv[str(row) + ', ' + str(col)]['x'] = {}
       shifts_csv[str(row) + ', ' + str(col)]['y'] = {}
       shifts_csv[str(row) + ', ' + str(col)]['x'] = ox
       shifts_csv[str(row) + ', ' + str(col)]['y'] = oy
       cv2.circle(out, (int(ox), int(oy)), 5, (0, 255, 0), -1)
       cv2.putText(out, "center of die", (int(ox) - 25, int(oy) - 25),cv2.   FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
       die_centers.append(die_center)
       
       if(col != ideal_param['num_cols']):
           col += 1
       else:
           row += 1
           col = 0
       
    for i, p in enumerate(alignment_pts):
        shifts['alignment ' + str(i)] = {'x': p[0], 'y': p[1], 'theta': 0}
        shifts_csv['alignment ' + str(i)] = {'x': p[0], 'y': p[1], 'theta': 0}


     #sets up csv file
    with open(image_paths['scan_dir'] + '/edge_det.csv', 'w') as f:
        fields = ['pos', 'x', 'y', 'theta']
        w = csv.DictWriter(f, fields)
        w.writeheader()
        #key,value
        for k, v in sorted(shifts_csv.items()):
            row = {'pos': k}
            row.update(v)
            w.writerow(row)


    return out, cross_centers, square_centers, shifts_csv
    
        

In [970]:
def angle_misalign(input, die_contours, cross_centers, shifts_csv):
    cv2.drawContours(input, die_contours, -1, (0, 0, 255), 5)
    
    center_inside = []
    angles = []
    #TODO: order die_contours by r,c and make center_inside a dictionary according to that r,c
    for c in die_contours:
       for i in range(len(cross_centers)):
        inside = cv2.pointPolygonTest(c, cross_centers[i], False)
        if(inside == 1):
           center_inside.append(cross_centers[i]) #should add 2 centers to center_inside in order by die
   
   #iterate through center_inside by 2, taking each 2 and calculating the angle between them 
    print("length of centers_inside (should be 50)", len(center_inside))
    for i in range(0, len(center_inside)-1, 2):
       angle = get_angle_simple(center_inside[i], center_inside[i+1])
       angle2 = get_angle_simple(center_inside[i+1], center_inside[i])
       #account for angle calculation with incorrect orientation (CCW instead of CW?)
       if angle < -5:
          angles.append(angle+180)
       if angle2 < -5:
          angles.append(angle2+180)
       elif angle>5:
          angles.append(angle-180)
       elif angle2>5:
          angles.append(angle2-180)
       else:
          angles.append(angle)
          angles.append(angle2)
       if i + 1 < len(center_inside):  # Check if next element exists (for even length)
         print(f"angle: {angles[i]}")
         cv2.putText(input, f"angle: {angles[i]}", (center_inside[i][0] - 25, center_inside[i][1] + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 100, 100), 3)
         cv2.putText(input, f"angle: {angles[i+1]}", (center_inside[i+1][0] - 25, center_inside[i+1][1] + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 100, 100), 3)
         



          

In [971]:
def get_angle_simple(a, b):
    
    #Calculates the angle of a straight line (with respect to the horizon) between points a and b:
    return math.degrees(math.atan2(a[1] - b[1], a[0] - b[0]))

In [972]:
def identify_dies(contours, alignment_pts, min_die_area=20000, max_die_area=100000):
    #identify die contours
    #Identify dies with thresholding/contouring based on a minimum and maximum die area:
    nontrivial_contours = []
    nontrivial_offsets = []
    for i, c in enumerate(contours):
        area = cv2.contourArea(c)
        
        if area < min_die_area or area > max_die_area:
            continue
            #calculates the bounding rectangle of each contour, where x,y are top left corner
        x, y, w, h = cv2.boundingRect(c)
        
        #to track if an contour is near a alignment pt
        is_alignment_mark = False
        for alignment_pt in alignment_pts:
            #it calculates the distance between the contour's top-left corner (x, y) and the alignment point
            if math.dist([x, y], alignment_pt) < 300:
                is_alignment_mark = True
                break
        #If is_alignment_mark is True, it skips to the next contour in the loop
        if is_alignment_mark:
            continue
        #The contour (c) is appended to the nontrivial_contours list.
        #The top-left corner coordinates (x, y) of the contour's bounding rectangle are stored in nontrivial_offsets as an offset from the image origin.
        nontrivial_contours.append(c)
        nontrivial_offsets.append([x, y])
        #sorted based on top left coord
    nontrivial_contours.sort(key=lambda c: (cv2.boundingRect(c)[0], cv2.boundingRect(c)[1]))
    nontrivial_offsets.sort(key=lambda o: (o[0], o[1]))
    nontrivial_offsets = np.array(nontrivial_offsets)
    
    return nontrivial_contours, nontrivial_offsets

In [973]:
def order_alignment_marker_list(alignment_pts):
    
    #Orders the list of global alignment marker positions such that it is in the order LL, LR, UL, UR:
    new_list = alignment_pts.copy()
    sum = np.zeros((np.shape(alignment_pts)[0]), dtype='int')
    h = 0
    
    while h < 2:
        
        i = 2*h
        j = i + 1
        sum[i] = new_list[i][0] + new_list[i][1]
        sum[j] = new_list[j][0] + new_list[j][1]
        
        if sum[j] < sum[i]:
            
            new_list[i] = alignment_pts[j]
            new_list[j] = alignment_pts[i]
        
        h = h + 1
    
    alignment_pts = new_list.copy()
    
    return alignment_pts

In [974]:
def crop_rotated(img, h_old, w_old, angle): #h_old and w_old are before rotation and cropping
    
    #Rotates an image by a given angle... then crops the image to remove the rotational defects:
    a = abs(math.radians(angle))
    sin_a = abs(math.sin(a))
    h, w = img.shape[0], img.shape[1]
    dh, dw = h_old * sin_a, w_old * sin_a #defects of rotation?
    hc, wc = h - 2 * dh, w - 2 * dw #height and width after rotation and cropped


    return img[int((h-hc)/2):int((h+hc)/2), int((w-wc)/2):int((w+wc)/2)]

In [981]:
def calculate_scale_using_alignment_markers(alignment_pts, alignment_mark_dist):
    #how many pixels is how many microns, output is pixel to micron scale, usually half micron per pixel
    #Image scale is calculed using the global alignment marker coordinates and given marker to marker distance:
    d01 = np.absolute(alignment_pts[0] - alignment_pts[1])
    d02 = np.absolute(alignment_pts[0] - alignment_pts[2])
    d13 = np.absolute(alignment_pts[1] - alignment_pts[3])
    d23 = np.absolute(alignment_pts[2] - alignment_pts[3])
    
    dist01 = np.linalg.norm(d01)
    dist02 = np.linalg.norm(d02)
    dist13 = np.linalg.norm(d13)
    dist23 = np.linalg.norm(d23)
    
    max_vertical_dist = max(d01[0], d02[0], d13[0], d23[0])
    max_horizontal_dist = max(d01[1], d02[1], d13[1], d23[1])
    rescale_amount = max_horizontal_dist / max_vertical_dist
    avg_pixel_dist = (dist01 + dist02 + dist13 + dist23) / 4
    pixel_um_scale = alignment_mark_dist / avg_pixel_dist
    
    print('Pixel to um scale (w.r.t. alignment markers) [um/pixel]:', pixel_um_scale)
    
    return pixel_um_scale, rescale_amount

In [976]:
scan_dir = '/Users/travisha/Downloads/CHIPS_research'
image_paths = {}
image_paths['scan_dir'] = scan_dir
image_paths['input'] = scan_dir + '/v1_Sample2_Scan2_corrected.png'
image_paths['canny'] = scan_dir + '/canny.bmp'
image_paths['cross'] = scan_dir + '/cross_quarter.bmp'
image_paths['squares'] = scan_dir + '/squares_quarter.bmp'
image_paths['alignment'] = scan_dir + '/alignment_quarter.bmp'
ideal_param = {}
ideal_param['num_rows'] = 5
ideal_param['num_cols'] = 5
ideal_param['die_width'] = 1900
ideal_param['die_height'] = 1900
ideal_param['pitch'] = 2575
ideal_param['alignment_mark_dist'] = 25000
ideal_param['assembly_x_offset'] = -1175
ideal_param['assembly_y_offset'] = 540
scale_to_use='detected'
needs_rotation=True
has_defects=False

img = cv2.imread(image_paths['input'])
output = img.copy()
#grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#apply blur (helps remove noise in canny)
blur = cv2.GaussianBlur(gray, (5,5), 0)

#apply canny
canny = cv2.Canny(blur, threshold1=120, threshold2=200 ) #can tighten thresholds to get less noise, but might detect less]
cv2.imwrite(image_paths['canny'], canny)

#Identify global alignment markers, adjust rotation, and rescale dimensions:
template_alignment = cv2.imread(image_paths['alignment'])#template matching of large circles 
template_alignment_gray = cv2.cvtColor(template_alignment, cv2.COLOR_BGR2GRAY)
alignment_pts, result = match_template(output, gray, template_alignment_gray, "alignment", 0.9) #actual template matching of alignment markers (large circles at corners of entire image)
alignment_pts = order_alignment_marker_list(alignment_pts)
ideal_param['scale'], rescale = calculate_scale_using_alignment_markers(alignment_pts, ideal_param['alignment_mark_dist'])
ideal_param['min_die_area'] = (0.9*ideal_param['die_width']/ideal_param['scale'])*(0.9*ideal_param['die_height']/ideal_param['scale'])
ideal_param['max_die_area'] = (1.1*ideal_param['die_width']/ideal_param['scale'])*(1.1*ideal_param['die_height']/ideal_param['scale'])
#resizes based on pixel to micron factor and preserves the original aspect ratio by keeping the height the same and turn gray again
img = cv2.resize(img, (int(rescale*img.shape[1]), img.shape[0]), interpolation=cv2.INTER_CUBIC)
canny = cv2.resize(canny, (int(rescale*img.shape[1]), img.shape[0]), interpolation=cv2.INTER_CUBIC)
result = img.copy()
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)



#If the image needs rotation, rotate and crop the image, then re-identify global alignment markers and re-calculated scale:
#dont want to do any actual physical rotation of image, just want to use rotation in calculations
if needs_rotation:
    #gets angle btwn 2 points using x axis and line btwn pts
    theta = 0 - get_angle_simple(alignment_pts[1], alignment_pts[0])
    #entire image rotation
    print("Wafer scan rotation:", theta)
    h, w = img.shape[0], img.shape[1]
    img = ndimage.rotate(img, -theta)
    img = crop_rotated(img, h, w, -theta)
    result = img.copy()
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #alignment_pts recalculated with rotated image
    alignment_pts, result = match_template(result, img_gray, template_alignment_gray, "alignment", 0.9)
    alignment_pts = order_alignment_marker_list(alignment_pts)
    ideal_param['scale'], _ = calculate_scale_using_alignment_markers(alignment_pts, ideal_param['alignment_mark_dist'])
    ideal_param['min_die_area'] = (0.9*ideal_param['die_width']/ideal_param['scale'])*(0.9*ideal_param['die_height']/ideal_param['scale'])
    ideal_param['max_die_area'] = (1.1*ideal_param['die_width']/ideal_param['scale'])*(1.1*ideal_param['die_height']/ideal_param['scale'])

#Calculates midpoint of the global alignemnt markers, modifies alignment pts to be in center of large alginment circles: 
w0, h0 = template_alignment_gray.shape[::-1]#reverses order to get width first

for i, c in enumerate(alignment_pts):#enumerate takes an tierable object and returns an enumerate object, which is an iterator, outputs i which is index and c which is the alginment pt
    alignment_pts[i] = np.array([c[0] + w0 // 2, c[1] + h0 // 2])#TODO:since c[1] is top left y coord, shouldnt u subtract half the height?

 #Use CV2 thresholding to get all contours in the image: HOW WE IDENTIFY THE DIES
#pixels above threshold 105 gets converted to maxval 255 or uses OTSU's where threshold is calculated?, bw is thersholded image, removes background from dies
_, bw = cv2.threshold(gray, 105, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#uses thresholded image to find contours
#returns a list of all contours (dies) in image, each contour is a array of x,y boundary points of the object (can draw using cv2.drawContours)
contours, _ = cv2.findContours(bw, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]#this slices, -2 is second to last element, means take everything from the findContours output starting from second to last output e.g contours

#Identify dies by searching for contours with area matching the expected die area such that min area = 0.81*die area and max area = 1.21*die area:
    #The contour (c) is appended to the nontrivial_contours list.
    #The top-left corner coordinates (x, y) of the contour's bounding rectangle are stored in nontrivial_offsets as an offset from the image origin.
die_contours, die_offsets = identify_dies(contours, alignment_pts, min_die_area=ideal_param['min_die_area'], max_die_area=ideal_param['max_die_area'])


template_cross = cv2.imread(image_paths['cross'])
template_cross_gray = cv2.cvtColor(template_cross, cv2.COLOR_BGR2GRAY)

template_squares = cv2.imread(image_paths['squares'])
template_squares_gray = cv2.cvtColor(template_squares, cv2.COLOR_BGR2GRAY)
#properly detects all crosses
cross_pts, output = match_template(output, gray, template_cross_gray, "cross", 0.7)
# #TODO: locate location  of square corner markers
# num_matches = 0
# iter = 0
# max_iter = 20
# target = 50
thres = 0.778
# while num_matches != target and iter <= max_iter:
squares_pts, output = match_template(output, gray, template_squares_gray, "squares", thres)
#     num_matches = len(squares_pts)
#     if(num_matches > target):
#         thres += .001
    
#     if(num_matches < target):
#         thres -= .001
        
#     if(num_matches == target):
#         print('IDEAL THRESHOLD FOR DETECTING SQUARES CORNER ALIGNMENT MARKERS:', thres)
#     iter += 1

# print(squares_pts)
# cross_pts = [
#     [7161, 2064],
#     [8437, 2077],
#     [6392, 2084],
#     [5149, 2089],
#     [7668, 2092],
#     [4602, 2094],
#     [3369, 2107],
#     [5918, 2107],
#     [2600, 2131],
#     [3833, 2144],
#     [8480, 3323],
#     [7710, 3335],
#     [6453, 3355],
#     [7223, 3358],
#     [5929, 3364],
#     [3931, 3382],
#     [4701, 3386],
#     [5160, 3386],
#     [3404, 3408],
#     [2634, 3420],
#     [8499, 4578],
#     [7730, 4591],
#     [6469, 4623],
#     [7238, 4624],
#     [4695, 4634],
#     [5926, 4639],
#     [3925, 4641],
#     [3421, 4654],
#     [5157, 4663],
#     [2652, 4681],
#     [8485, 5848],
#     [7255, 5875],
#     [6485, 5877],
#     [5969, 5886],
#     [7716, 5887],
#     [3937, 5910],
#     [4707, 5911],
#     [5200, 5912],
#     [3441, 5950],
#     [2671, 5957],
#     [8507, 7116],
#     [7253, 7140],
#     [7738, 7144],
#     [6483, 7168],
#     [4724, 7175],
#     [5997, 7179],
#     [5227, 7183],
#     [3441, 7192],
#     [3955, 7194],
#     [2672, 7211]
# ]
# square_pts = [
#     [7183, 2833], [8454, 2846], [6413, 2854], [5129, 2858], [7684, 2862],
#     [3393, 2877], [5899, 2878], [2624, 2901], [3883, 2913], [8492, 4094],
#     [7723, 4106], [6451, 4126], [7221, 4129], [5952, 4134], [3927, 4153],
#     [5182, 4156], [4697, 4157], [3416, 4178], [2646, 4190], [8512, 5349],
#     [7742, 5361], [6468, 5393], [7237, 5395], [4702, 5405], [5950, 5409],
#     [3932, 5412], [3447, 5425], [5180, 5433], [2678, 5451], [8524, 6617],
#     [7258, 6646], [6488, 6648], [5996, 6656], [7755, 6657], [3937, 6681],
#     [4707, 6682], [5227, 6683], [3448, 6720], [2679, 6728], [4024, 7229],
#     [8536, 7886], [7281, 7910], [7766, 7914], [6512, 7938], [4743, 7946],
#     [6001, 7950], [5231, 7954], [3460, 7963], [3974, 7965], [2690, 7981]
# ]


out, cross_contours, square_contours = truncate_contour(cross_pts, square_pts, canny, template_cross_gray, image_paths, result)
for x,y in square_pts:
    cv2.circle(out, (x,y), 3, (0,0,255), -1)

final, cross_centers, square_centers, shifts_csv = centroid(die_contours, cross_contours, square_contours, alignment_pts, image_paths, ideal_param, out)
#angle_misalign(final, cross_contours, cross_centers, shifts_csv)

output_filename = f"{image_paths['scan_dir']}/centroids.png"
cv2.imwrite(output_filename, final)#for testing centroid and angle
output_filename = f"{image_paths['scan_dir']}/cornermarkers.png"
cv2.imwrite(output_filename, out)#for testing truncate_contour


Detected alignment markers: 4
Pixel to um scale (w.r.t. alignment markers) [um/pixel]: 2.207990242824598
Wafer scan rotation: 0.12149643463822107
Detected alignment markers: 4
Pixel to um scale (w.r.t. alignment markers) [um/pixel]: 2.207017789296064
center (np.float64(7202.5), np.float64(2856.0))
center (np.float64(8489.5), np.float64(2868.75))
center (np.float64(6446.5), np.float64(2876.5))
center (np.float64(5160.5), np.float64(2880.5))
center (np.float64(7719.0), np.float64(2884.5))
center (np.float64(3421.25), np.float64(2899.5))
center (np.float64(5925.666666666667), np.float64(2905.0))
center (np.float64(2651.0), np.float64(2923.5))
center (np.float64(3911.25), np.float64(2935.5))
center (np.float64(nan), np.float64(nan))
center (np.float64(7757.75), np.float64(4128.5))
center (np.float64(6479.333333333333), np.float64(4153.333333333333))
center (np.float64(7269.0), np.float64(4151.5))
center (np.float64(5979.0), np.float64(4161.333333333333))
center (np.float64(3960.33333333333

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


True

In [977]:
bruh = 1
bruh

1